In [1]:
# [Cell 1: .env 로드 및 Agent 임포트]
# .env 파일 로드 (core.py가 로드하지만, 노트북 환경을 위해 한 번 더)
import sys
import os
from dotenv import load_dotenv

# 1. 현재 프로젝트 루트 디렉토리(jesafe/)의 절대 경로를 찾습니다.
# os.getcwd() (현재 폴더)의 부모 폴더('..')가 프로젝트 루트라고 지정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. 이 경로를 파이썬 모듈 검색 경로(sys.path)에 추가합니다.
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"프로젝트 루트 경로 추가: {project_root}")

load_dotenv()
print(".env 로드 완료:", os.getenv("DATABASE_URL") is not None)

프로젝트 루트 경로 추가: /Users/lwj0831/jesafe
.env 로드 완료: True


In [2]:
# [Cell 2: 메인 Agent 동기 테스트 (RunnableWithMessageHistory 사용)]
from src.services import agent_with_history
from src.data_loader import _load_jeju_coords

# 테스트 전에 CSV 데이터를 메모리에 로드합니다.
_load_jeju_coords()

# 테스트에 사용할 고유 세션 ID
session_id = "test_session_notebook_123"

# --- 1. 날씨 질문 테스트 ---
# (프롬프트 규칙에 따라 '제주도'가 '제주시'로 변환되어야 함)
query1 = "오늘 제주도 날씨 어때?"
print(f"User: {query1}")

response1 = agent_with_history.invoke(
    {"input": query1},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response1['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 2. 사고 통계 질문 테스트 ---
query2 = "제주시에서 낙상 사고가 몇 건이야?"
print(f"User: {query2}")

response2 = agent_with_history.invoke(
    {"input": query2},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response2['output']}\n")
# (agent_with_history가 이 대화를 session_id에 자동 저장)


# --- 3. (중요) 대화 맥락(Context) 질문 테스트 ---
# (이전 질문의 '제주시'를 기억하고 '제주시' 날씨를 답해야 함)
query3 = "그럼 그 지역 날씨는 어때?"
print(f"User: {query3}")

response3 = agent_with_history.invoke(
    {"input": query3},
    config={"configurable": {"session_id": session_id}}
)
print(f"AI: {response3['output']}\n")

core로부터 PGVector Store 가져오는 중...
Initializing PGVector connection to collection: tourism_docs...
Initializing CacheBackedEmbeddings...
BM25 리트리버 초기화 중... (문서 로드)
BM25 인덱스 생성을 위해 문서 로드 중...
BM25 인덱스 생성을 위해 문서 로드 중... (기준 경로: /Users/lwj0831/jesafe/data/tourism_docs)


/Users/lwj0831/jesafe/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


BM25: 총 4개 페이지 로드 완료.
BM25 리트리버 초기화 완료.
✅ [data_loader] 제주도 (KMA 격자) DB 로드 완료 (경로: /Users/lwj0831/jesafe/data/kma_jeju_grid_info.csv)
User: 오늘 제주도 날씨 어때?

Invoking: `get_current_weather` with `제주시`


[Tool] '오늘 날씨 (OWM)' 도구 호출됨: 제주시
'제주시'의 오늘 날씨 요약: Currently, the weather is clear with a temperature of 15°C and a slight breeze with a wind speed of 10 meters per second. The humidity is at 65%, and the visibility is at 10000 meters. The air pressure is at 1020 hPa. It feels like 14°C outside with a dew point of 9°C. There are no clouds in the sky, and the UV index is at 0. Overall, it's a pleasant evening with clear skies and comfortable temperatures, but it might be a bit windy, so you might want to hold onto your hats if you're heading out.

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


현재 제주시의 날씨는 맑고 기온은 15°C입니다. 바람은 약간 불고 있으며 풍속은 초당 10미터입니다. 습도는 65%이고, 가시거리는 10000미터입니다. 기압은 1020 hPa입니다. 체감 온도는 14°C이며, 이슬점은 9°C입니다. 하늘은 구름 한 점 없이 맑고, 자외선 지수는 0입니다. 전반적으로 날씨가 매우 쾌적하지만 바람이 조금 부는 편이니 외출 시 모자를 잘 챙기시길 바랍니다.

> Finished chain.
AI: 현재 제주시의 날씨는 맑고 기온은 15°C입니다. 바람은 약간 불고 있으며 풍속은 초당 10미터입니다. 습도는 65%이고, 가시거리는 10000미터입니다. 기압은 1020 hPa입니다. 체감 온도는 14°C이며, 이슬점은 9°C입니다. 하늘은 구름 한 점 없이 맑고, 자외선 지수는 0입니다. 전반적으로 날씨가 매우 쾌적하지만 바람이 조금 부는 편이니 외출 시 모자를 잘 챙기시길 바랍니다.

User: 제주시에서 낙상 사고가 몇 건이야?


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `jeju_safety_statistics_db` with `제주시에서 낙상 사고가 몇 건이야?`


[Tool] SQL Agent 도구 호출됨: 제주시에서 낙상 사고가 몇 건이야?
Action: sql_db_list_tables
Action Input: jeju_accidentsI need to check the schema of the "jeju_accidents" table to understand what columns are available and relevant to the question about fall accidents in Jeju City.
Action: sql_db_schema
Action Input: jeju_accidents
CREATE TABLE jeju_accidents (
	"SEASN_NM" TEXT, 
	"TRFC_ACDNT_SE_NM" TEXT, 
	"ETC_TRFC_ACDNT_NM" TEXT, 
	"ACDNT_OCRN_LOT" DOUBLE PRECISION, 
	"ACDNT_OCRN_LAT" DOUBLE PRECISION, 
	"DCLR_MM" BIGINT, 
	"SRIL_ONCR_NM" TEXT, 
	"PTN_OCRN_TYPE_NM" TEXT, 
	"PTN_SYM_SE_NM" TEXT, 
	"ACDNT_OCRN_PLC_NM" TEXT, 
	"GRNDS_SGG_NM" TEXT, 
	"ACDNT_INJR_NM" TEXT, 
	"DCLR_YR" BIGINT
)

/*
3 rows from jeju_accidents table:
SEASN_NM	TRFC_ACDNT_SE_NM	ETC_TRFC_ACDNT_NM	ACDNT_OCRN_LOT	ACDNT_OCRN_LAT	DCLR_MM	SRIL_ONCR_NM	PTN_OCRN_TYPE_NM	PTN_SYM_SE_NM	ACDNT_OCRN_PLC_NM	GRNDS_SGG_NM	ACDNT_INJR_NM	DCLR_YR
겨울	알수없음	알수없음	126.48464829	33.488695

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


제주시에서 발생한 낙상 사고는 총 390건입니다.

> Finished chain.
AI: 제주시에서 발생한 낙상 사고는 총 390건입니다.

User: 그럼 그 지역 날씨는 어때?

Invoking: `get_current_weather` with `제주시`


[Tool] '오늘 날씨 (OWM)' 도구 호출됨: 제주시
'제주시'의 오늘 날씨 요약: Currently, the weather is clear with a temperature of 15°C and a slight breeze with a wind speed of 10 meters per second. The humidity is at 65%, and the visibility is at 10000 meters. The air pressure is at 1020 hPa. It feels like 14°C outside with a dew point of 9°C. There are no clouds in the sky, and the UV index is at 0. Overall, it's a pleasant evening with clear skies and comfortable temperatures, but it might be a bit windy, so you might want to hold onto your hats if you're heading out.현재 제주시의 날씨는 맑고 기온은 15°C입니다. 바람은 약간 불고 있으며 풍속은 초당 10미터입니다. 습도는 65%이고, 가시거리는 10000미터입니다. 기압은 1020 hPa입니다. 체감 온도는 14°C이며, 이슬점은 9°C입니다. 하늘은 구름 한 점 없이 맑고, 자외선 지수는 0입니다. 전반적으로 날씨가 매우 쾌적하지만 바람이 조금 부는 편이니 외출 시 모자를 잘 챙기시길 바랍니다.

> Finished chain.
AI: 현재 제주시의 날씨는 맑고 기온은 15°C입니다. 바람은 약간 불고 있으며 풍속은 초당 10미터입니다. 습도

In [ ]:
# [Cell 3: RAG 임베딩 및 리트리버 전체 테스트]
import sys
import os

# (Cell 1과 동일) 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.core import get_vector_store, get_bm25_retriever
from src.tools import ensemble_retriever 

print("Connecting to Vector Store & Initializing Retrievers...")
try:
    # 앱 시작 시점과 동일하게 리소스 초기화
    vector_store = get_vector_store()
    bm25_retriever = get_bm25_retriever()
    
    # 테스트 쿼리
    query = "제주도 오름 추천해줘"
    print(f"\nTest Query: '{query}'")
    
    # -----------------------------------------------
    # 테스트 1: PGVector (의미기반 검색)
    # -----------------------------------------------
    print("\n--- 1. PGVector (Similarity Search) Test ---")
    vector_results = vector_store.similarity_search(query, k=3)
    print(f"Found {len(vector_results)} Chunks:")
    for i, doc in enumerate(vector_results):
        print(f"  [Vec {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 2: BM25 (키워드 기반 검색)
    # -----------------------------------------------
    print("\n--- 2. BM25 (Keyword Search) Test ---")
    bm25_results = bm25_retriever.invoke(query) 
    print(f"Found {len(bm25_results)} Chunks:")
    for i, doc in enumerate(bm25_results):
        print(f"  [BM25 {i+1}] {doc.page_content[:100]}...")

    # -----------------------------------------------
    # 테스트 3: Ensemble (최종 앙상블)
    # -----------------------------------------------
    print("\n--- 3. Ensemble Retriever Test ---")
    ensemble_results = ensemble_retriever.invoke(query)
    print(f"Found {len(ensemble_results)} Chunks (after re-ranking):")
    for i, doc in enumerate(ensemble_results):
        print(f"  [Ensemble {i+1}] (Source: {doc.metadata.get('source', 'N/A')})")
        print(f"    {doc.page_content[:200]}...")

except Exception as e:
    print(f"\n❌ 테스트 실패: {e}")

ImportError: cannot import name 'get_bm25_retriever' from 'src.core' (/Users/lwj0831/jesafe/src/core.py)